1. 数据清洗

In [1]:
import pandas as pd
import json

# 读取
path = "./data/healthcare_dataset.csv"

df = pd.read_csv(path)


# 选取要保存的列
cols_keep = ['Name','Age','Gender','Blood Type','Medical Condition','Date of Admission', 'Doctor','Insurance Provider', 'Billing Amount','Hospital','Discharge Date','Medication','Test Results']
df = df[cols_keep].copy()

# 原始数据量
print('original amount', len(df))

# 删除缺失关键字的数据
df = df.dropna(subset = cols_keep)
print('After deleting na rows', len(df))

df.to_csv("./data/cleaned_hc_dataset.csv")

original amount 55500
After deleting na rows 55500


2. 提取实体和关系


In [1]:
import spacy
import pandas as pd
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span

# 加载模型
nlp = spacy.load("en_core_sci_sm")

In [13]:
# 自定义术语库
TEST_TERMS = ["Normal", "Abnormal", "Inconclusive"]
ADMISSION_TERMS = ["Emergency", "Elective", "Urgent"]

# Initialize Matcher
matcher = Matcher(nlp.vocab)
phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

patterns = {
    "TEST": TEST_TERMS,
    "ADMISSION": ADMISSION_TERMS
}

for label, terms in patterns.items():
    phrase_matcher.add(label, [nlp.make_doc(term) for term in terms])

In [16]:
def extract_entities(row):
    entities = []

    entities.append((row["Name"], "PATIENT"))
    entities.append((row["Hospital"], "HOSPITAL"))
    entities.append((row["Insurance Provider"], "INSURANCE"))
    entities.append((row["Doctor"], "DOCTOR"))
    entities.append((row["Blood Type"], "BLOOD"))
    entities.append((row["Gender"], "GENDER"))
    entities.append((row["Medical Condition"], "DISEASE"))
    entities.append((row["Billing Amount"], "BILL"))
    entities.append((row["Test Results"], "TEST"))


    return list(set(entities))


In [ ]:
df = pd.read_csv("./data/cleaned_hc_dataset.csv")
df["entities"] = df.apply(extract_entities, axis=1)

In [21]:
def extract_relations(row):
    relations = []
    
    # Patient - Hospital
    relations.append((row["Name"], "TREATED_AT", row["Hospital"]))

    # Patient - Doctor
    relations.append((row["Name"], "TREATED_BY", row["Doctor"]))

    # Patient - Insurance
    relations.append((row["Name"], "COVERED_BY", row["Insurance Provider"]))

    # Patient - Bill
    relations.append((row["Name"], "PAY", row["Billing Amount"]))

    # Patient - GENDER
    relations.append((row["Name"], "IS", row["Gender"]))

    # Patient - Result
    relations.append((row["Name"], "DIAGNOSED_AS", row["Medical Condition"]))

    # Medical Condition - Test Results
    relations.append((row["Medical Condition"], "HAS_TEST_RESULT", row["Test Results"]))

    return relations

In [22]:
df["relations"] = df.apply(extract_relations, axis=1)

In [ ]:
# 保存结果
df.to_csv("./data/hc_dataset_with_entities_relations.csv", index=False)

# 打印部分结果进行验证
print(df[["entities", "relations"]].head())

                                            entities  \
0  [(B-, BLOOD), (Matthew Smith, DOCTOR), (Cancer...   
1  [(Medicare, INSURANCE), (Samantha Davies, DOCT...   
2  [(Obesity, DISEASE), (Female, GENDER), (27955....   
3  [(Abnormal, TEST), (Medicare, INSURANCE), (Her...   
4  [(adrIENNE bEll, PATIENT), (14238.317813937623...   

                                           relations  
0  [(Bobby JacksOn, TREATED_AT, Sons and Miller),...  
1  [(LesLie TErRy, TREATED_AT, Kim Inc), (LesLie ...  
2  [(DaNnY sMitH, TREATED_AT, Cook PLC), (DaNnY s...  
3  [(andrEw waTtS, TREATED_AT, Hernandez Rogers a...  
4  [(adrIENNE bEll, TREATED_AT, White-White), (ad...  


### 3. 用Neo4j构建图谱并可视化关键节点

In [24]:
from neo4j import GraphDatabase
import pandas as pd

# 连接到本地 Neo4j 数据库
URI = "bolt://localhost:7687" 
USER = "neo4j"
PASSWORD = "password"

# 连接数据库
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

In [ ]:
# 读取提取的实体和关系
df_spacy = pd.read_csv("./data/hc_dataset_with_entities_relations.csv")

import ast
df_spacy["entities"] = df_spacy["entities"].apply(ast.literal_eval)
df_spacy["relations"] = df_spacy["relations"].apply(ast.literal_eval)

In [27]:
# 为了缩短处理时间，只展示前200行数据
df_spacy_subset = df_spacy.head(200)

In [ ]:
# 创建节点
def create_nodes(tx, entities):
    for entity, label in entities:
        tx.run(f"MERGE (n:{label} {{name: $name}})", name=entity)

# 处理数据
with driver.session() as session:
    for _, row in df_spacy_subset.iterrows():
        session.write_transaction(create_nodes, row["entities"])

In [ ]:
def batch_create_relations(tx, relation_list):
    query = """
    UNWIND $relations AS rel
    MATCH (a {name: rel.src}), (b {name: rel.dst})
    MERGE (a)-[:`REL_TYPE` {type: rel.rel_type}]->(b)
    """
    tx.run(query, relations=relation_list)


batch_size = 100  # 每批 500 组数据
with driver.session() as session:
    for i in range(0, len(df_spacy_subset), batch_size):
        batch_relations = df_spacy_subset["relations"].iloc[i : i + batch_size].explode().dropna().tolist()


        formatted_relations = [
            {"src": rel[0], "rel_type": rel[1], "dst": rel[2]} for rel in batch_relations
        ]

        session.write_transaction(batch_create_relations, formatted_relations)
